### Clustering_LDA_4models
TESTA 4 VERSÕES DISTINTAS DE MODELAGEM DE TÓPICOS A PARTIR DE DOCUMENTOS DA BASE NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

In [ ]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import pickle

In [25]:
outputs = "../outputs/"

In [26]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Especifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows)
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
else:
    encoding_type = 'ISO-8859-1'
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'

### Seleciona textos da base mysql para fazer modelagem de tópicos

In [ ]:
count = 0
texts = []
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ") # filtra textos   
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        if row is None: break

        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')

        text =  row[4]
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)

### Cria lista de stopwords

In [6]:
# load nltk's stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            nltk.corpus.stopwords.words('german') + \
            additional_words

stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [7]:
%time texts = [[word for word in text if word not in stopwords] for text in texts]

Wall time: 0 ns


In [8]:
print(len(texts[0]))

97


In [9]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [10]:
len(corpus)

2

#### Salva dictionary e corpus com o pickle

In [29]:
file_corpus = '../outputs/LDAcorpus.pkl'
file_dictionary = '../outputs/LDAdictionary.pkl'

In [30]:
''' caso queira carregar os arquivos '''
corpus = pickle.load(open(file_corpus, 'rb'))
dictionary = pickle.load(open(file_dictionary, 'rb'))

### Teste com 20 tópicos.

In [10]:
%time lda20 = models.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=50)
pickle.dump(lda20, open('../outputs/model_lda_20.pkl', 'wb'))

CPU times: user 3h 13min 56s, sys: 14h 6min 32s, total: 17h 20min 29s
Wall time: 1h 13min 11s


In [11]:
lda20.print_topics(-1, num_words=5)

[(0,
  '0.015*chanceler + 0.013*telegrama + 0.012*recebido + 0.011*serie + 0.007*brasil'),
 (1,
  '0.028*telegrama + 0.018*minuta + 0.010*página + 0.008*distribuição + 0.008*silveira'),
 (2,
  '0.042*militar + 0.020*guiana + 0.019*acordo + 0.017*forças + 0.015*militares'),
 (3, '0.017*artigo + 0.017*art + 0.011*lei + 0.008*federal + 0.006*república'),
 (4, '0.055*chile + 0.039*peru + 0.026*santiago + 0.020*buenos + 0.020*aires'),
 (5,
  '0.011*governo + 0.010*acordo + 0.009*ministério + 0.008*relações + 0.007*brasileiro'),
 (6,
  '0.008*comissão + 0.007*geral + 0.007*desenvolvimento + 0.006*internacional + 0.005*grupo'),
 (7,
  '0.044*senhor + 0.036*ministro + 0.021*presidente + 0.021*senhora + 0.016*república'),
 (8,
  '0.012*us + 0.010*brasil + 0.010*produtos + 0.007*mercado + 0.007*exportações'),
 (9,
  '0.020*presidente + 0.020*brasil + 0.018*governo + 0.014*ministro + 0.011*secreto'),
 (10, '0.011*rio + 0.010*paulo + 0.007*sr + 0.007*nacional + 0.007*portugal'),
 (11,
  '0.030*bra

#### Visualizando os tópicos com o PyLDAvis (20 tópicos)

In [12]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda20, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
18     19.240510        1       1 -0.172915  0.098474
6      10.769842        1       2 -0.172096  0.091252
9       9.288493        1       3 -0.083435 -0.176531
0       8.165891        1       4 -0.115040 -0.047129
5       6.728286        1       5 -0.178420 -0.004291
8       5.696177        1       6 -0.157214  0.105644
1       4.990426        1       7  0.141746 -0.125019
11      4.680461        1       8 -0.120699 -0.125176
10      3.740224        1       9 -0.015029 -0.174193
13      3.719465        1      10 -0.091610  0.100536
12      3.701514        1      11  0.098555  0.016605
14      3.471050        1      12  0.173121  0.134943
7       3.218071        1      13  0.089048 -0.310439
15      3.206771        1      14 -0.039923  0.160175
3       3.197836        1      15 -0.080077 -0.051992
16      2.477797        1      16 -0.122017  0.114601
17      2.115359        1      17  0.123608 -0.063449
2       0.653822        1      18  0.162824  0.063259
4       0.598649        1      19  0.285797  0.105471
19      0.339356        1      20  0.273776  0.087258, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
92208  Default  18672.000000        senhor  18672.000000  30.0000  30.0000
45702  Default  21114.000000      ministro  21114.000000  29.0000  29.0000
72749  Default  13791.000000     telegrama  13791.000000  28.0000  28.0000
20303  Default  21859.000000    presidente  21859.000000  27.0000  27.0000
62378  Default  12799.000000        acordo  12799.000000  26.0000  26.0000
80618  Default  48913.000000        brasil  48913.000000  25.0000  25.0000
8681   Default  10646.000000    embaixador  10646.000000  24.0000  24.0000
56290  Default  10445.000000    cooperação  10445.000000  23.0000  23.0000
39972  Default   5740.000000       nuclear   5740.000000  22.0000  22.0000
29625  Default  12320.000000    excelência  12320.000000  21.0000  21.0000
31690  Default  22367.000000      relações  22367.000000  20.0000  20.0000
27276  Default  23288.000000       governo  23288.000000  19.0000  19.0000
36947  Default   5817.000000        itaipu   5817.000000  18.0000  18.0000
11965  Default  12242.000000     república  12242.000000  17.0000  17.0000
2060   Default   7605.000000     argentina   7605.000000  16.0000  16.0000
71084  Default   9704.000000           rio   9704.000000  15.0000  15.0000
30722  Default   9553.000000        visita   9553.000000  14.0000  14.0000
51415  Default   8311.000000        unidos   8311.000000  13.0000  13.0000
5763   Default   4910.000000        minuta   4910.000000  12.0000  12.0000
77061  Default   6581.000000        artigo   6581.000000  11.0000  11.0000
7486   Default   9125.000000     chanceler   9125.000000  10.0000  10.0000
10773  Default   3963.000000        corpus   3963.000000   9.0000   9.0000
31460  Default   9142.000000         geral   9142.000000   8.0000   8.0000
88166  Default  16161.000000    exteriores  16161.000000   7.0000   7.0000
52612  Default   8129.000000       projeto   8129.000000   6.0000   6.0000
80539  Default   2798.000000       militar   2798.000000   5.0000   5.0000
95620  Default   2808.000000         costa   2808.000000   4.0000   4.0000
27522  Default   3676.000000       senhora   3676.000000   3.0000   3.0000
357    Default   7866.000000      comissão   7866.000000   2.0000   2.0000
11975  Default   5331.000000       energia   5331.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
32313  Topic20     24.664976  leiteribeiro     25.465606   5.6539  -6.6116
18715  Topic20     24.247699        senhon     25.048329   5.6534  -6.6287
75766  Topic20     23.565282        beiden     24.365913   5.6525  -6.6573
55499  Topic20     22.828113            zr     23.628743   5.6514  -6

In [13]:
data_ldavis = pyLDAvis.gensim.prepare(lda20, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_20topics.html'))

### Teste com 30 tópicos.

In [14]:
%time lda30 = models.LdaModel(corpus, num_topics=30, id2word=dictionary, passes=50)
pickle.dump(lda30, open('../outputs/model_lda_30.pkl', 'wb'))

CPU times: user 4h 1min 7s, sys: 18h 31min 13s, total: 22h 32min 21s
Wall time: 1h 25min 10s


In [15]:
lda30.print_topics(-1, num_words=5)

[(0,
  '0.030*brasil + 0.019*cooperação + 0.016*ministro + 0.015*relações + 0.009*visita'),
 (1,
  '0.036*presidente + 0.015*ministro + 0.014*embaixador + 0.014*governo + 0.013*disse'),
 (2,
  '0.018*chanceler + 0.015*serie + 0.014*brasil + 0.011*recebido + 0.010*telegrama'),
 (3,
  '0.010*desenvolvimento + 0.008*brasil + 0.007*unidos + 0.007*internacional + 0.005*nuclear'),
 (4, '0.025*itaipu + 0.018*corpus + 0.016*rio + 0.011*energia + 0.010*cota'),
 (5,
  '0.020*governo + 0.017*brasil + 0.012*relações + 0.011*exteriores + 0.011*brasileiro'),
 (6,
  '0.013*us + 0.011*produtos + 0.009*brasil + 0.008*mercado + 0.008*exportações'),
 (7,
  '0.022*comissão + 0.019*geral + 0.018*projeto + 0.016*assembleia + 0.015*resolução'),
 (8,
  '0.114*excelência + 0.088*senhor + 0.039*república + 0.039*presidente + 0.028*embaixador'),
 (9,
  '0.012*brasil + 0.011*política + 0.009*relações + 0.007*mundo + 0.005*governo'),
 (10,
  '0.069*chile + 0.051*peru + 0.033*bolívia + 0.029*santiago + 0.024*saudit

#### Visualizando os tópicos com o PyLDAvis (30 tópicos)

In [16]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda30, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      14.028070        1       1 -0.229190 -0.113038
5      10.281547        1       2 -0.198872  0.036765
9       8.545966        1       3 -0.228651 -0.053434
0       6.807095        1       4 -0.163914  0.096722
13      6.333190        1       5 -0.172446 -0.067975
2       5.643500        1       6 -0.181244  0.046797
28      5.263369        1       7  0.063807 -0.029647
6       5.057063        1       8 -0.188618 -0.127438
26      4.834599        1       9 -0.183518 -0.041320
7       4.159777        1      10 -0.126872 -0.043432
22      3.412828        1      11  0.065735  0.241089
1       2.918981        1      12 -0.108834  0.189993
20      2.552971        1      13  0.004153  0.165443
4       2.525378        1      14 -0.125815 -0.133927
29      2.130023        1      15 -0.079745  0.007942
12      2.110510        1      16  0.070556  0.110882
21      2.087682        1      17 -0.067221 -0.096367
14      1.889198        1      18  0.159126 -0.173892
17      1.523688        1      19  0.187172 -0.148392
23      1.448159        1      20  0.051539 -0.080831
25      1.369641        1      21  0.054779  0.072573
8       1.116001        1      22  0.116921  0.244210
24      1.110075        1      23  0.145496  0.100553
27      0.918832        1      24  0.126329  0.158269
16      0.647142        1      25  0.069874  0.171229
10      0.446388        1      26  0.173435 -0.100016
19      0.398539        1      27  0.179079 -0.098221
18      0.216366        1      28  0.204173 -0.118657
15      0.198380        1      29  0.205465 -0.066779
11      0.025044        1      30  0.177304 -0.149102, topic_info=      Category          Freq           Term         Total  loglift  logprob
term                                                                       
92208  Default  18869.000000         senhor  18869.000000  30.0000  30.0000
20303  Default  21889.000000     presidente  21889.000000  29.0000  29.0000
29625  Default  12285.000000     excelência  12285.000000  28.0000  28.0000
72749  Default  13345.000000      telegrama  13345.000000  27.0000  27.0000
45702  Default  21670.000000       ministro  21670.000000  26.0000  26.0000
31690  Default  22786.000000       relações  22786.000000  25.0000  25.0000
80618  Default  48880.000000         brasil  48880.000000  24.0000  24.0000
11965  Default  12279.000000      república  12279.000000  23.0000  23.0000
88166  Default  16287.000000     exteriores  16287.000000  22.0000  22.0000
27276  Default  23281.000000        governo  23281.000000  21.0000  21.0000
94341  Default   7011.000000        secreto   7011.000000  20.0000  20.0000
30722  Default   9528.000000         visita   9528.000000  19.0000  19.0000
8681   Default  10546.000000     embaixador  10546.000000  18.0000  18.0000
67484  Default   8052.000000       silveira   8052.000000  17.0000  17.0000
84198  Default   4984.000000     informação   4984.000000  16.0000  16.0000
56290  Default  10801.000000     cooperação  10801.000000  15.0000  15.0000
7486   Default   9086.000000      chanceler   9086.000000  14.0000  14.0000
2060   Default   7388.000000      argentina   7388.000000  13.0000  13.0000
5763   Default   4572.000000         minuta   4572.000000  12.0000  12.0000
4758   Default   5442.000000        azeredo   5442.000000  11.0000  11.0000
36947  Default   5737.000000         itaipu   5737.000000  10.0000  10.0000
8197   Default   3780.000000         global   3780.000000   9.0000   9.0000
357    Default   7529.000000       comissão   7529.000000   8.0000   8.0000
71084  Default   9724.000000            rio   9724.000000   7.0000   7.0000
51415  Default   8560.000000         unidos   8560.000000   6.0000   6.0000
20423  Default   4406.000000   confidencial   4406.000000   5.0000   5.0000
64152  Default  12853.000000       política  12853.000000   4.0000   4.0000
4693   Default   2894.000

In [17]:
data_ldavis = pyLDAvis.gensim.prepare(lda30, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_30topics.html'))

### Teste com 40 tópicos.

In [18]:
%time lda40 = models.LdaModel(corpus, num_topics=40, id2word=dictionary, passes=50)
pickle.dump(lda40, open('../outputs/model_lda_40.pkl', 'wb'))

CPU times: user 4h 50min 10s, sys: 22h 22min 42s, total: 1d 3h 12min 53s
Wall time: 1h 37min 22s


In [19]:
lda40.print_topics(-1, num_words=5)

[(0, '0.038*artigo + 0.020*art + 0.010*lei + 0.008*presente + 0.008*federal'),
 (1,
  '0.019*pana + 0.017*cônsul + 0.014*apreciacao + 0.010*morris + 0.009*éxteriores'),
 (2,
  '0.013*recebido + 0.012*pagina + 0.012*petróleo + 0.011*serah + 0.010*tel'),
 (3,
  '0.021*united + 0.011*brazil + 0.010*state + 0.009*minister + 0.008*states'),
 (4,
  "0.013*deux + 0.013*d'une + 0.011*gouvernement + 0.010*council + 0.010*committee"),
 (5,
  '0.078*telegrama + 0.039*minuta + 0.022*página + 0.017*distribuição + 0.017*retransmito'),
 (6,
  '0.035*senhor + 0.033*senhora + 0.026*ministro + 0.019*presidente + 0.015*república'),
 (7, '0.038*cuba + 0.033*oea + 0.025*humanos + 0.025*direitos + 0.022*chile'),
 (8,
  '0.007*international + 0.007*states + 0.006*government + 0.006*nations + 0.005*brazil'),
 (9, '0.022*paulo + 0.021*sr + 0.015*carlos + 0.013*venezuela + 0.012*dr'),
 (10,
  '0.052*nuclear + 0.028*acordo + 0.022*brasil + 0.022*energia + 0.020*nucleares'),
 (11,
  '0.007*governo + 0.006*preside

#### Visualizando os tópicos com o PyLDAvis (40 tópicos)

In [20]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda40, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
11     12.731759        1       1  0.144602  0.135528
22     10.483708        1       2  0.165261  0.151164
21      8.468086        1       3  0.238568  0.158073
39      6.602644        1       4  0.229752  0.082990
12      5.883153        1       5  0.166097  0.171003
15      5.394495        1       6  0.233460 -0.026295
37      4.547241        1       7  0.200209  0.111124
19      3.985767        1       8  0.198283 -0.091730
13      3.530365        1       9  0.139692 -0.186316
29      3.525729        1      10 -0.030547 -0.091032
36      3.429941        1      11  0.231283  0.175309
17      3.356611        1      12  0.192169 -0.027827
0       2.636218        1      13  0.139639  0.044964
8       2.578087        1      14 -0.103207  0.049834
5       2.106224        1      15  0.000456 -0.131133
38      1.987612        1      16  0.115296 -0.050507
10      1.875953        1      17  0.160400  0.125150
16      1.820848        1      18  0.089245 -0.201941
18      1.820016        1      19  0.050693 -0.211512
6       1.780211        1      20  0.047694 -0.263202
20      1.768893        1      21 -0.012027 -0.005424
2       1.637957        1      22  0.105371  0.022152
3       1.246261        1      23 -0.111434 -0.026540
9       1.190392        1      24 -0.024461 -0.159500
7       0.996540        1      25 -0.006668  0.027430
33      0.968728        1      26 -0.013300 -0.116190
25      0.826729        1      27 -0.035529 -0.224389
26      0.750252        1      28 -0.034777 -0.061199
24      0.367823        1      29 -0.171985 -0.004492
34      0.243773        1      30 -0.185892  0.027081
35      0.219556        1      31 -0.191837  0.025012
32      0.215064        1      32 -0.196743  0.036620
30      0.185748        1      33 -0.187426  0.033223
31      0.168107        1      34 -0.206973  0.059770
4       0.162341        1      35 -0.213448  0.063501
28      0.161104        1      36 -0.207773  0.058167
27      0.132845        1      37 -0.215987  0.056901
1       0.099221        1      38 -0.227194  0.075983
23      0.061342        1      39 -0.234420  0.094207
14      0.052655        1      40 -0.236542  0.094042, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
92208  Default  19048.000000        senhor  19048.000000  30.0000  30.0000
72749  Default  13480.000000     telegrama  13480.000000  29.0000  29.0000
20303  Default  21297.000000    presidente  21297.000000  28.0000  28.0000
45702  Default  21534.000000      ministro  21534.000000  27.0000  27.0000
80618  Default  49315.000000        brasil  49315.000000  26.0000  26.0000
29625  Default  12426.000000    excelência  12426.000000  25.0000  25.0000
8681   Default  10822.000000    embaixador  10822.000000  24.0000  24.0000
67484  Default   8127.000000      silveira   8127.000000  23.0000  23.0000
31690  Default  23176.000000      relações  23176.000000  22.0000  22.0000
30722  Default   9379.000000        visita   9379.000000  21.0000  21.0000
39972  Default   5763.000000       nuclear   5763.000000  20.0000  20.0000
94341  Default   6857.000000       secreto   6857.000000  19.0000  19.0000
88166  Default  16338.000000    exteriores  16338.000000  18.0000  18.0000
62378  Default  13070.000000        acordo  13070.000000  17.0000  17.0000
77061  Default   6600.000000        artigo   6600.000000  16.0000  16.0000
27276  Default  23212.000000       governo  23212.000000  15.0000  15.0000
56290  Default  11173.000000    cooperação  11173.000000  14.0000  14.0000
11965  Default  12256.000000     república  12256.000000  13.0000  13.0000
99403  Default   5508.000000         ordem   5508.000000  12.0000  12.0000
7486   Default   9063.000000     chanceler   9063.000000  11.0000  11.0000
64152  Default  12960.000000      polít

In [21]:
data_ldavis = pyLDAvis.gensim.prepare(lda40, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_40topics.html'))

### Teste com 45 tópicos.

In [22]:
%time lda45 = models.LdaModel(corpus, num_topics=45, id2word=dictionary, passes=50)
pickle.dump(lda45, open('../outputs/model_lda_45.pkl', 'wb'))

CPU times: user 4h 53min 58s, sys: 22h 1min 36s, total: 1d 2h 55min 34s
Wall time: 1h 36min 27s


In [23]:
lda45.print_topics(-1, num_words=5)

[(0,
  '0.040*angola + 0.032*áfrica + 0.031*sul + 0.021*portugal + 0.017*governo'),
 (1,
  '0.013*política + 0.009*desenvolvimento + 0.008*relações + 0.007*internacional + 0.005*brasil'),
 (2, '0.014*uruguay + 0.009*pc + 0.009*mw + 0.007*porcento + 0.006*hum'),
 (3,
  '0.071*canal + 0.058*panamá + 0.031*congresso + 0.019*chefes + 0.017*tratado'),
 (4, '0.025*cuba + 0.023*oea + 0.018*reunião + 0.017*chile + 0.013*peru'),
 (5,
  '0.044*monsieur + 0.026*instituto + 0.024*carreira + 0.022*curso + 0.020*diplomata'),
 (6,
  '0.020*recebido + 0.017*g/sg + 0.015*telegrama + 0.014*página + 0.011*due'),
 (7,
  '0.015*madame + 0.015*leurs + 0.011*brésil + 0.010*excellences + 0.010*pays'),
 (8,
  '0.008*developing + 0.006*development + 0.006*committee + 0.005*energy + 0.005*national'),
 (9,
  '0.147*frança + 0.122*paris + 0.095*francesa + 0.060*giscard + 0.049*francês'),
 (10,
  '0.064*aaaaa + 0.016*brazext + 0.013*konsultationen + 0.011*beide + 0.010*editores'),
 (11,
  '0.011*brasil + 0.009*gove

#### Visualizando os tópicos com o PyLDAvis (45 tópicos)

In [24]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda45, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
11     11.519600        1       1 -0.214453  0.136312
21     11.106259        1       2 -0.203633  0.092895
1       9.919170        1       3 -0.234460  0.195109
27      6.332326        1       4 -0.222819  0.114882
37      5.863841        1       5 -0.214558  0.014252
20      4.896089        1       6 -0.192521 -0.132684
26      4.641020        1       7 -0.177959  0.136456
12      4.174114        1       8 -0.261936  0.028711
15      3.564942        1       9 -0.139635  0.007556
38      3.016559        1      10 -0.131456  0.092767
39      3.012306        1      11  0.048323 -0.074641
18      2.587912        1      12 -0.101895  0.039855
41      2.310034        1      13  0.080644  0.025947
19      2.218827        1      14 -0.083067 -0.107370
17      2.002576        1      15 -0.103180 -0.209127
33      1.908790        1      16 -0.213057 -0.016329
34      1.774073        1      17 -0.160567  0.115311
40      1.709796        1      18 -0.066628 -0.254274
44      1.634396        1      19 -0.071209 -0.280222
31      1.495637        1      20  0.008303 -0.207352
16      1.380207        1      21  0.028665 -0.008511
35      1.264367        1      22  0.043083 -0.141812
4       1.256557        1      23 -0.043212  0.031450
42      1.251898        1      24  0.001171 -0.074225
6       1.223670        1      25  0.029736 -0.055260
8       1.000295        1      26  0.125230  0.042571
0       0.989963        1      27  0.043343 -0.015816
13      0.645344        1      28 -0.031287  0.122892
36      0.634055        1      29  0.063201 -0.004090
29      0.593155        1      30  0.087514 -0.156372
28      0.507417        1      31  0.133740  0.037903
24      0.475072        1      32  0.099824 -0.024102
30      0.465157        1      33  0.079572 -0.048966
7       0.397366        1      34  0.142274  0.026718
32      0.391846        1      35  0.112592 -0.027862
2       0.377099        1      36  0.155955 -0.004600
14      0.318186        1      37  0.135765 -0.083121
5       0.286311        1      38  0.080861  0.004135
22      0.217616        1      39  0.184148  0.045641
3       0.178630        1      40  0.164765  0.094974
9       0.162259        1      41  0.175865  0.058663
25      0.121545        1      42  0.201683  0.105825
43      0.085905        1      43  0.211998  0.107873
23      0.056498        1      44  0.214489  0.119449
10      0.031319        1      45  0.214792  0.128588, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
45702  Default  21629.000000         ministro  21629.000000  30.0000  30.0000
72749  Default  12926.000000        telegrama  12926.000000  29.0000  29.0000
92208  Default  18869.000000           senhor  18869.000000  28.0000  28.0000
8681   Default  10833.000000       embaixador  10833.000000  27.0000  27.0000
20303  Default  21367.000000       presidente  21367.000000  26.0000  26.0000
31690  Default  22850.000000         relações  22850.000000  25.0000  25.0000
27276  Default  23135.000000          governo  23135.000000  24.0000  24.0000
51415  Default   8477.000000           unidos   8477.000000  23.0000  23.0000
80618  Default  49348.000000           brasil  49348.000000  22.0000  22.0000
29625  Default  12544.000000       excelência  12544.000000  21.0000  21.0000
88166  Default  16223.000000       exteriores  16223.000000  20.0000  20.0000
71084  Default   9868.000000              rio   9868.000000  19.0000  19.0000
56290  Default  10954.000000       cooperação  10954.000000  18.0000  18.0000
67484  Default   8170.000000         silveira   8170.000000  17.0000  17.0000
62378  Default  12961.000000           acordo  12961.000000  16.0000  16.0000
7486   Default   9075.000000        chanceler   9075.000000  15.0000  15.0000
39972  Default 

In [25]:
data_ldavis = pyLDAvis.gensim.prepare(lda45, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_45topics.html'))

### Teste com random_state 0 (60 tópicos)

In [ ]:
corpus = corpus[0:20]

%time lda60_00 = models.LdaModel(corpus, num_topics=60, id2word=dictionary, passes=50, eval_every=1, random_state=0)
pickle.dump(lda60_00, open('../outputs/model_lda_60_rs_00.pkl', 'wb'))

#### Visualizando os tópicos com o PyLDAvis (random_state 0, 60 tópicos)

In [ ]:
pyLDAvis.enable_notebook()
#pyLDAvis.gensim.prepare(lda60_00, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda60_00, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_60topics_rs00.html'))

### Teste com random_state 1 (60 tópicos)

In [ ]:
%time lda60_01 = models.LdaModel(corpus, num_topics=60, id2word=dictionary, passes=50, eval_every=1, random_state=1)
pickle.dump(lda60_01, open('../outputs/model_lda_60_rs_01.pkl', 'wb'))

#### Visualizando os tópicos com o PyLDAvis (random_state 1, 60 tópicos)

In [ ]:
pyLDAvis.enable_notebook()
#pyLDAvis.gensim.prepare(lda60_01, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda60_01, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_60topics_rs01.html'))

### Teste com random_state 0 (100 tópicos)

In [ ]:
%time lda100_00 = models.LdaModel(corpus, num_topics=100, id2word=dictionary, passes=50, eval_every=1, random_state=0)
pickle.dump(lda100_00, open('../outputs/model_lda_100_rs_00.pkl', 'wb'))

#### Visualizando os tópicos com o PyLDAvis (random_state 0, 100 tópicos)

In [ ]:
pyLDAvis.enable_notebook()
#pyLDAvis.gensim.prepare(lda100_00, corpus, dictionary)

In [ ]:
data_ldavis = pyLDAvis.gensim.prepare(lda100_00, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_100topics_rs00.html'))